In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import r2_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch_explain as te
from torch.nn.functional import one_hot
from torch_explain.logic.metrics import test_explanation, complexity, concept_consistency, formula_consistency
from torch_explain.logic.nn import entropy, psi
from torch_explain.logic.utils import replace_names

In [2]:
n_splits = 5
test_size = 0.2
random_state = 42
shuffle = True
flip = False

In [5]:
x1 = pd.read_csv('../../data/node_vs_nodenn_lens/u.csv', index_col=None, header=0)
x2 = pd.read_csv('../../data/node_vs_nodenn_lens/u equation only.csv', index_col=None, header=0)
x = pd.concat((x1, x2), axis=0).reset_index()
x = x.iloc[:, 1:]
feature_names = x.columns
class_names = ['nnode', 'equations']
x = x.values
y = np.zeros(500)
y[len(y)//2:] = 1
# y = y.values
# if flip:
#     t = x
#     x = y
#     y = t
x.shape, y.shape

((500, 20), (500,))

In [6]:
layers = [
    te.nn.EntropyLinear(x.shape[1], 20, n_classes=2, temperature=0.3),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(20, 10),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(10, 1),
]
model = torch.nn.Sequential(*layers)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_form = torch.nn.BCEWithLogitsLoss()
model.train()

Sequential(
  (0): EntropyLinear(
    in_features=20, out_features=20, n_classes=2
    (conceptizator): Conceptizator(activation=identity_bool, threshold=0.5)
  )
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=20, out_features=10, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=10, out_features=1, bias=True)
)

In [7]:
r2_train_list, r2_test_list = [], []
columns=['Class name', 'Test model accuracy', 'Explanation', 'Test explanation accuracy', 'Explanation complexity']
results = pd.DataFrame(columns=columns)
sss = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
for split, (train_index, test_index) in enumerate(sss.split(x, y)):
    print(f"Fold {split+1}/{n_splits} \n\t Train...")
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    scaler_x = StandardScaler()
    scaler_x.fit(x_train)
    x_train = scaler_x.transform(x_train)
    x_test = scaler_x.transform(x_test)
    
    x_train = torch.FloatTensor(x_train)
    x_test = torch.FloatTensor(x_test)
    y_train = one_hot(torch.LongTensor(y_train)).to(torch.float)
    y_test = one_hot(torch.LongTensor(y_test)).to(torch.float)
    x2 = torch.concat((x_train, x_test))
    y2 = torch.concat((y_train, y_test))
#     model = RandomForestClassifier(random_state=random_state)
#     model = LinearRegression()
#     model = MLPRegressor()
#     model.fit(x_train, y_train)
    layers = [
        te.nn.EntropyLinear(x.shape[1], 10, n_classes=2, temperature=0.9),
        torch.nn.LeakyReLU(),
#         torch.nn.Linear(50, 10),
#         torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 1),
    ]
    model = torch.nn.Sequential(*layers)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)
    loss_form = torch.nn.BCEWithLogitsLoss()
    model.train()

#     train_mask = torch.where(torch.FloatTensor(train_index))[0]
#     test_mask = torch.where(torch.FloatTensor(test_index))[0]
    train_mask = train_index
    test_mask = test_index
    for epoch in range(2001):
        # train step
        optimizer.zero_grad()
        y_pred = model(x2).squeeze(-1)
        loss = loss_form(y_pred[train_mask], y2[train_mask]) + 0.0001 * te.nn.functional.entropy_logic_loss(model)
        loss.backward()
        optimizer.step()

#         # compute accuracy
#         if epoch % 100 == 0:
    train_auc = roc_auc_score(y2[train_mask], torch.sigmoid(y_pred[train_mask]).detach().numpy())
    test_auc = roc_auc_score(y2[test_mask], torch.sigmoid(y_pred[test_mask]).detach().numpy())
    print(f'Epoch {epoch}: loss {loss:.4f} train accuracy: {train_auc:.4f} test accuracy: {test_auc:.4f}')
    
    explanations, local_exp = entropy.explain_classes(model, x2, y2, train_mask, test_mask,
                                                  c_threshold=0., y_threshold=0., verbose=True,
                                                  concept_names=feature_names, class_names=class_names,
                                                  topk_explanations=10, max_minterm_complexity=5,
                                                  material=True, good_bad_terms=True, 
                                                  max_accuracy=True)
    
    c0 = pd.DataFrame([[
        class_names[0], 
        test_auc, 
        explanations['0']['explanation'], 
        explanations['0']['explanation_accuracy'],
        explanations['0']['explanation_complexity'],
    ]], columns=columns)
    
    c1 = pd.DataFrame([[
        class_names[1], 
        test_auc, 
        explanations['1']['explanation'], 
        explanations['1']['explanation_accuracy'],
        explanations['1']['explanation_complexity'],
    ]], columns=columns)
    
    results = pd.concat((results, c0, c1))

Fold 1/5 
	 Train...
Epoch 2000: loss 0.0222 train accuracy: 1.0000 test accuracy: 1.0000
Explanation class nnode: (CD4 nai & B act & ~B memory) | (CD8 act & B act & ~pro) - acc. = 0.9100 - compl. = 6.0000
Explanation class equations: (virus LRT & ~B act) | (APC act & ~APC nai LRT & ~B nai & ~B act) - acc. = 0.7300 - compl. = 6.0000
Fold 2/5 
	 Train...
Epoch 2000: loss 0.0194 train accuracy: 1.0000 test accuracy: 1.0000
Explanation class nnode: B act & short lived plasma & ~virus LRT - acc. = 0.7700 - compl. = 3.0000
Explanation class equations: (virus LRT & pro) | (pro & ~APC nai LRT) | (~APC nai LRT & ~B act) - acc. = 0.7000 - compl. = 6.0000
Fold 3/5 
	 Train...
Epoch 2000: loss 0.0180 train accuracy: 1.0000 test accuracy: 0.9992
Explanation class nnode: B act & ~inf cell LRT & ~virus LRT - acc. = 0.9600 - compl. = 3.0000
Explanation class equations: (virus LRT & ~B act & ~short lived plasma) | (~virus LRT & ~APC nai LRT & ~B nai & ~B act) | (~virus LRT & ~APC nai LRT & ~B nai & ~s

In [8]:
results.to_csv('lens_results_nnode_ode.csv')

In [9]:
results

,Class name,Test model accuracy,Explanation,Test explanation accuracy,Explanation complexity
0,nnode,1.000000,(CD4 nai & B act & ~B memory) | (CD8 act & B a...,0.91,6
0,equations,1.000000,(virus LRT & ~B act) | (APC act & ~APC nai LRT...,0.73,6
0,nnode,1.000000,B act & short lived plasma & ~virus LRT,0.77,3
0,equations,1.000000,(virus LRT & pro) | (pro & ~APC nai LRT) | (~A...,0.70,6
0,nnode,0.999199,B act & ~inf cell LRT & ~virus LRT,0.96,3
0,equations,0.999199,(virus LRT & ~B act & ~short lived plasma) | (...,0.77,11
0,nnode,1.000000,B act,0.82,1
0,equations,1.000000,(virus LRT & ~B act) | (B memory & ~CD8 nai & ...,0.74,6
0,nnode,0.999800,APC nai LRT | (B act & Ab),0.69,3
0,equations,0.999800,time & APC act & ~CD8 nai & ~B nai & ~B act,0.92,5
